In [4]:
import torch
import pandas as pd
import pyaging as pya
import os

In [5]:
os.system("curl -o coefficients.csv https://www.aging-us.com/article/102049/supplementary/SD2/0/aging-v11i12-102049-supplementary-material-SD2.csv")

0

In [8]:
df = pd.read_csv('coefficients.csv')

df['feature'] = df['CpGs']
df['coefficient'] = df['Coefficient_refined_RPC']

df = df[df.coefficient != 0]

In [9]:
features = df['feature'][1:].tolist()

weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]])

In [10]:
model = pya.models.LinearModel(len(features))

model.linear.weight.data = weights
model.linear.bias.data = intercept

model

LinearModel(
  (linear): Linear(in_features=395, out_features=1, bias=True)
)

In [11]:
weights_dict = {
    'preprocessing': None, 
    'preprocessing_helper': None,
    'postprocessing': None,
    'postprocessing_helper': None,
    'features': features,
    'weight_dict': model.state_dict(),
}

metadata_dict = {
    'species': 'Homo sapiens',
    'data_type': 'methylation',
    'year': 2019,
    'implementation_approved_by_author(s)': '⌛',
    'preprocessing': weights_dict['preprocessing'], 
    'postprocessing': weights_dict['postprocessing'], 
    'citation': "Lee, Yunsung, et al. \"Placental epigenetic clocks: estimating gestational age using placental DNA methylation levels.\" Aging (Albany NY) 11.12 (2019): 4238.",
    'doi': "https://doi.org/10.18632/aging.102049",
    "notes": None,
}

In [12]:
torch.save(weights_dict, '../weights/leerefinedrpc.pt')
torch.save(metadata_dict, '../metadata/leerefinedrpc.pt')

In [13]:
os.system("rm coefficients.csv")

0